In [1]:
# Standard Libraries
import pandas as pd
import numpy as np
import os
import sys
from datetime import date

# Image Libraries
import cv2
from cv2 import imread, imshow, waitKey, destroyAllWindows, rectangle, CascadeClassifier

# Database
import psycopg2

# Visualizations
from matplotlib import pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Database

In [2]:
con = None

try:
    
    con = psycopg2.connect(database='kennedy', user='postgres',
        password='')

    cur = con.cursor()
    cur.execute('SELECT version()')

    version = cur.fetchone()[0]
    print(version)

except psycopg2.DatabaseError as e:

    print('Error {}'.format(e))
    sys.exit(1)

finally:

    if con:
        con.close()

PostgreSQL 12.1 on x86_64-apple-darwin16.7.0, compiled by Apple LLVM version 8.1.0 (clang-802.0.42), 64-bit


In [21]:
def create_database():
    con = None
    try:
        con = psycopg2.connect(database='kennedy', user='postgres', 
                               password='')

        with con:
            cur = con.cursor()

            cur.execute("DROP TABLE IF EXISTS images")
            cur.execute("CREATE TABLE images(id BIGINT DEFAULT 0 NOT NULL, dir_path TEXT, date DATE DEFAULT '1/1/1970' NOT NULL, img_orig BYTEA DEFAULT 'NaN' NOT NULL, face BOOLEAN DEFAULT 'False' NOT NULL, img_proc BYTEA DEFAULT 'NaN', face_class INTEGER DEFAULT 0 NOT NULL)")
            cur.execute("COMMENT ON COLUMN images.id IS E'This number will come from a counter programatically.'")
            cur.execute("COMMENT ON COLUMN images.dir_path IS E'directory path to original image.  This is so that you can trace back to the imported data for files with the same names in multiple directories.'")
            cur.execute("COMMENT ON COLUMN images.date IS E'date image was imported.'")
            cur.execute("COMMENT ON COLUMN images.face_class IS E'Classes based off _# in filename.  0,1,2,3,4,5. 0=no_class, 1=commecicomm,2=smile,3=shut,4=shocked,5=sunglasses.'")
            cur.execute("COMMENT ON COLUMN images.img_proc IS E'Processed images!'")
            cur.execute("ALTER TABLE images ADD CONSTRAINT dir_path PRIMARY KEY(id)")
            cur.execute("ALTER TABLE images ADD COLUMN eyes BOOLEAN DEFAULT 'False' NOT NULL")
            cur.execute("ALTER TABLE images ADD COLUMN mouth BOOLEAN DEFAULT 'False' NOT NULL")
            cur.execute("ALTER TABLE images ADD COLUMN nose BOOLEAN DEFAULT 'False' NOT NULL")
            con.commit()    
        
    except psycopg2.DatabaseError as e:

        print('Error {}'.format(e))
        sys.exit(1)
    
#     finally:

#         if con:
#             con.close()
        

In [22]:
def insert_database(id_, file_name, dir_path):
#     con = psycopg2.connect(database='kennedy', user='postgres', 
#                                password='')
    with con:
        cur = con.cursor()
        today = date.today().strftime('%m/%d/%Y')
        imgopen = open(dir_path, 'rb').read()
        filename = (file_name)
        data = (id_, dir_path, today, psycopg2.Binary(imgopen) , False, 0)
        query = "INSERT INTO images(id, dir_path, date, img_orig, face, face_class) VALUES (%s, %s, %s, %s, %s, %s)"
        cur.execute(query, data)
#         if con:
#             con.close()
    return

# Preprocess Files for Storage

In [13]:
file_name = '' # image name
file_path = '' # path to file

In [14]:
# Process Images Directory for Feeding into the Model
def build_image_list(path):
    '''
    Search (path) and make a recursive listing of paths to each file.
    INPUTS: path = top of directory tree to begin recursion for images. ex) "../data"
    OUTPUTS: fname = A full path listing of all files in the directory tree.  astype.list()
    '''
    fname = []
    for root,d_names,f_names in os.walk(path):
        f_names = [f for f in f_names if not f[0] == '.'] # skip hidden files: .DSstore
        d_names = [d for d in d_names if not d[0] == '.'] # skip hidden folders: .git
        for f in f_names:
            fname.append(os.path.join(root, f))
    return fname

In [15]:
def process_images(fname):
    i = 0
    for file in fname:
        global classifier
        global file_name
        global file_path
        file_name = os.path.basename(file)
        file_path = file
        print('{}...{}...{}'.format(i, file_name, file_path))
        insert_database(i, file_name, file_path) 
        i+=1
    return

In [16]:
image_list = build_image_list('data/TD_RGB_E')
print(len(image_list))
print(image_list[1])

560
data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_4.jpg


In [17]:
print(image_list)

['data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_5.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_4.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_1.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_3.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_2.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/92/TD_RGB_E_5.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/92/TD_RGB_E_4.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/92/TD_RGB_E_1.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/92/TD_RGB_E_3.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/92/TD_RGB_E_2.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/104/TD_RGB_E_5.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/104/TD_RGB_E_4.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/104/TD_RGB_E_1.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/104/TD_RGB_E_3.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/104/TD_RGB_E_2.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/103/TD_RGB_E_5.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/103/TD_RGB_E_4.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/103/TD_RGB_E_1.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/103/TD_RGB_E_3.jpg', 'data/TD_RGB_E/TD_RGB_E_Set4/103/TD_RGB_E_2.jpg', 'data/TD_

# Main

In [23]:
create_database()

In [24]:
process_images(image_list)

0...TD_RGB_E_5.jpg...data/TD_RGB_E/TD_RGB_E_Set4/95/TD_RGB_E_5.jpg


InterfaceError: connection already closed

In [ ]:
for path in image_list:
    new_string = string.replace("/", "--")
    new_strings.append(new_string)

In [ ]:
cur.execute("SELECT dir_path, img_orig FROM images")
for row in cur:
    img = cur.fetchone()
#    open('Folder/'+row[1], 'wb').write(str(imgc[0])) write binary to updated column

# Crappy Model

In [ ]:
%timeit
def detect_face(img_rgb):
    '''
    1st classifier: HAAR classifier for face detection
    from mod 4
    if faces detected, update feature from False to True
    read image from original column
    draw box around face
    write image to modified field
    call detect_eyes
    if no face detected, exit loop
    '''
    global classifier
    global file_name
    bounding_boxes = classifier.detectMultiScale(img_rgb, scaleFactor=1.3, minNeighbors=5, minSize=(150, 150))
    box_list = []
    for box in bounding_boxes:
        box_list.append(box)
    if len(box_list) < 1:
        img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
        cv2.imwrite('../sort/no_face/'+file_name+'.jpg', img_rgb)
    else:
        draw_boxes(img_rgb, box_list)
    return

In [ ]:
%timeit
def detect_eyes():
    '''
    1st classifier: HAAR classifier for face detection
    from mod 4
    if faces detected, update feature from False to True
    read image from original column
    draw box around face
    write image to modified field
    call detect_eyes
    if no face detected, exit loop
    '''
    pass

In [ ]:
%timeit
def detect_mouth():
    '''
    1st classifier: HAAR classifier for face detection
    from mod 4
    if faces detected, update feature from False to True
    read image from original column
    draw box around face
    write image to modified field
    call detect_eyes
    if no face detected, exit loop
    '''
    pass

In [ ]:
%timeit
def detect_nose():
    '''
    1st classifier: HAAR classifier for face detection
    from mod 4
    if faces detected, update feature from False to True
    read image from original column
    draw box around face
    write image to modified field
    call detect_eyes
    if no face detected, exit loop
    '''
    pass

## softmax to classify emotion

In [ ]:
%timeit
def predict_emotion():
    '''
    1st classifier: HAAR classifier for face detection
    from mod 4
    if faces detected, update feature from False to True
    read image from original column
    draw box around face
    write image to modified field
    call detect_eyes
    if no face detected, exit loop
    '''
    pass

In [ ]:
def draw_boxes(img_rgb, box_list):
    for box in box_list:
        x, y, width, height = box
        x2, y2 = x + width, y + height
        cv2.rectangle(img_rgb, (x, y), (x2, y2), (0,255,0), 3)
    save_image(img_rgb)
    return

In [ ]:
def save_image(img_rgb):
    global file_name
#     plt.imshow(img)
#     plt.xticks([]), plt.yticks([])
#     plt.show()
    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB)
    cv2.imwrite('../sort/face/'+file_name+'.jpg', img_rgb)
    return

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
cm = confusion_matrix(y_test, y_pred_class)

tn = cm_2[0,0]
fn = cm_2[0,1]
fp = cm_2[1,0]
tp = cm_2[1,1]

accurracy = (tp + tn)/(tn+tp+fn+fp)

precision = tp / (tp+fp)

recall = tp / (tp + fn)
f1_score = 2*precision*recall/(precision+recall)
print('My model metrics were: Accurracy: {}, Precision: {}, Recall: {}, and F1: {}'.format(accurracy,precision,recall,f1_score))


plt.matshow(cm)

plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show();

